# 1. Sleep well (35 points)

In [ ]:
import pandas as pd

# Load the data
X_train = pd.read_csv('data\Sleep-EDF-15_U-Time/X_train.csv', header=None)
y_train = pd.read_csv('data\Sleep-EDF-15_U-Time/y_train.csv', header=None)
X_test = pd.read_csv('data\Sleep-EDF-15_U-Time/X_test.csv', header=None)
y_test = pd.read_csv('data\Sleep-EDF-15_U-Time/y_test.csv', header=None)

## 1.1 Data understanding and preprocessing

In [ ]:
import matplotlib.pyplot as plt

# Calculate class frequencies for training data, normalized to [0,1]
class_frequencies = y_train.value_counts(normalize=True).sort_index()

# Print each class frequency for training data
print("Class frequencies for training data:")
for label, freq in class_frequencies.items():
    print(f"Class {label}: {freq*100:.2f}%")

# Plotting
labels = [f"Class {i}" for i in range(0, 5)]
freqs = class_frequencies.values * 100

plt.figure(figsize=(4, 3))
plt.bar(labels, freqs, color=plt.cm.tab10.colors, alpha=0.5)
plt.ylabel('Frequency (%)')
plt.title('Class Frequencies for Training Data')
plt.ylim(0, 100)
plt.grid(axis='y')

plt.savefig('output/class_frequencies.png')
plt.show()

## 1.2 Classification

### Method to evaluate the models

In [ ]:
from sklearn.metrics import classification_report, accuracy_score, zero_one_loss

def evaluate_classifier(y_true_train, y_pred_train, y_true_test, y_pred_test, classifier_name):
    train_accuracy = accuracy_score(y_true_train, y_pred_train)
    train_report = classification_report(y_true_train, y_pred_train)

    test_accuracy = accuracy_score(y_true_test, y_pred_test)
    test_report = classification_report(y_true_test, y_pred_test)

    print(f"\nEvaluation Metrics for {classifier_name}:\n")

    print(f"Train Accuracy: {train_accuracy:.2f}")
    print(f"Train Error: {zero_one_loss(y_true_train, y_pred_train):.2f}")
    print(f"Train Classification Report:\n{train_report}")

    print(f"Test Accuracy: {test_accuracy:.2f}")
    print(f"Test Error: {zero_one_loss(y_true_test, y_pred_test):.2f}")
    print(f"Test Classification Report:\n{test_report}")

### Multi-nominal Logistic Regression

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

# Hyperparameter tuning
param_grid = {
    'C': [0.0001, 0.001, 0.01, 0.1]
}

# Grid search
grid_search = GridSearchCV(estimator=LogisticRegression(max_iter=10000, n_jobs=-1),
                           param_grid=param_grid,
                           n_jobs=-1,
                           cv=5,
                           return_train_score=True)
grid_search.fit(X_train, y_train.values.ravel())

best_model = grid_search.best_estimator_

# Print best hyperparameters
print("Best Hyperparameters:")
print(grid_search.best_params_)

# Predict and evaluate using the best model
train_pred = best_model.predict(X_train)
test_pred = best_model.predict(X_test)

# Evaluate the model
evaluate_classifier(y_train, train_pred, y_test, test_pred, f"Logistic Regression (Best Model)")

# Plotting cross-validation results
mean_train_scores = grid_search.cv_results_['mean_train_score']
mean_test_scores = grid_search.cv_results_['mean_test_score']

# Bar chart settings
index = np.arange(len(param_grid['C']))
bar_width = 0.25

plt.figure(figsize=(5, 4))
plt.bar(index, 100-mean_train_scores*100, bar_width, label='Training Error', color='tab:blue', alpha=0.5)
plt.bar(index + bar_width, 100-mean_test_scores*100, bar_width, label='Test Error', color='tab:red', alpha=0.5)

# Set the x-tick positions and labels
tick_positions = index + bar_width/2
plt.xticks(tick_positions, param_grid['C'])

plt.xlabel('C (Inverse Regularization Strength)')
plt.ylabel('Error (%)')
plt.ylim(0, 22)
plt.title('Logistic Regression\nMean Error vs. C (Inverse Regularization Strength)')
plt.legend()
plt.grid(True, which='both')
plt.tight_layout()
plt.show()

### Random Forests

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt

# Hyperparameter tuning
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt']
}

# Grid search with return_train_score set to True
grid_search = GridSearchCV(estimator=RandomForestClassifier(n_jobs=-1), 
                           param_grid=param_grid, 
                           n_jobs=-1, 
                           cv=5,
                           verbose=3,
                           return_train_score=True)
grid_search.fit(X_train, y_train.values.ravel())

# Get the best estimator (model)
best_model = grid_search.best_estimator_

# Print the best hyperparameters found
print("Best Hyperparameters:")

print(grid_search.best_params_)

# Predict and evaluate using the best model
train_pred = best_model.predict(X_train)
test_pred = best_model.predict(X_test)

# Evaluate the model
evaluate_classifier(y_train, train_pred, y_test, test_pred, f"Random Forest (Best Model)")

# Plotting cross-validation results
mean_train_scores = grid_search.cv_results_['mean_train_score']
mean_test_scores = grid_search.cv_results_['mean_test_score']

# Bar chart settings
index = np.arange(len(param_grid['n_estimators']))
bar_width = 0.25

plt.figure(figsize=(3, 3))
plt.bar(index, 100-mean_train_scores*100, bar_width, label='Training Error', color='tab:blue', alpha=0.5)
plt.bar(index + bar_width, 100-mean_test_scores*100, bar_width, label='Test Error', color='tab:red', alpha=0.5)

# Set the x-tick positions and labels
tick_positions = index + bar_width/2
plt.xticks(tick_positions, param_grid['n_estimators'])

plt.xlabel('Number of Trees')
plt.ylabel('Error (%)')
plt.ylim(0, 22)
plt.title('Random Forest\nMean Error vs. Number of Trees')
plt.legend()
plt.grid(True, which='both')
plt.tight_layout()
plt.show()

### k-Nearest-Neighbor Classification

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV

# Define the parameter grid
# Rule of thumb: sqrt(n) where n is the number of samples: https://quantdev.ssri.psu.edu/sites/qdev/files/kNN_tutorial.html
# 4 set in the training dataset and 1 set for validation. Total samples: 33724; validation samples: 33724/5 = 6745; training samples: 33724 - 6745 = 26979
# K-neighbors should be an odd value in KNN to avoid ties at decision boundaries
# where majority voting occurs. Using odd numbers helps prevent ties.
param_grid = {'n_neighbors': range(1, 130, 2)}  # Considering odd values from 1 to 129 neighbors

# Grid Search to find the best number of neighbors
grid_search = GridSearchCV(estimator=KNeighborsClassifier(n_jobs=-1),
                           param_grid=param_grid,
                           return_train_score=True,
                           n_jobs=-1,
                           cv=5)
grid_search.fit(X_train, y_train.values.ravel())

# Train the model with the best number of neighbors
best_model = grid_search.best_estimator_

# Predict and evaluate using the best model
train_pred = best_model.predict(X_train)
test_pred = best_model.predict(X_test)

# Print the best parameters
print("Best parameters:", grid_search.best_params_)

# Evaluate the model
evaluate_classifier(y_train, train_pred, y_test, test_pred, "K-Nearest Neighbors")

# Plot the cross-validation results
k_values = np.array(grid_search.cv_results_['param_n_neighbors'].data, dtype=float)
train_error = np.array(100 - grid_search.cv_results_['mean_train_score']*100, dtype=float)
test_error = np.array(100 - grid_search.cv_results_['mean_test_score']*100, dtype=float)
train_std = np.array(grid_search.cv_results_['std_train_score']*100, dtype=float)
test_std = np.array(grid_search.cv_results_['std_test_score']*100, dtype=float)

plt.figure(figsize=(5, 4))
plt.plot(k_values, train_error, label="Train error", color="tab:blue", alpha=0.6)
plt.fill_between(k_values, train_error - train_std, train_error + train_std, color="tab:blue", alpha=0.1)  # Variance for train error
plt.plot(k_values, test_error, label='Test error', color="tab:red", alpha=0.6)
plt.fill_between(k_values, test_error - test_std, test_error + test_std, color="tab:red", alpha=0.1)  # Variance for test error

plt.title('KNN\nMean Train- and Test Error vs. Number of Neighbors')
plt.xlabel('Number of Neighbors (k)')
plt.ylabel('Error (%)')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.savefig('output/knn.png')
plt.show()

# 2. Invariance and normalization (30 points)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Original data
data = {
    "Mr. Good":     np.array([47, 35]),
    "Mr. Bad":      np.array([22, 40]),
    "Mr. Unknown":  np.array([21, 36])
}

# Calculate means
mean_x = np.mean([data["Mr. Good"][0], data["Mr. Bad"][0]])
mean_y = np.mean([data["Mr. Good"][1], data["Mr. Bad"][1]])

print(f"Mean of age: {mean_x}")
print(f"Mean of income: {mean_y}\n")

# Center the data (zero mean)
centered_data = {key: value - [mean_x, mean_y] for key, value in data.items()}

for key, value in centered_data.items():
    print(f"Centered coordinates for {key}: {value}")

# Calculate variances
var_x = np.var([centered_data["Mr. Good"][0], centered_data["Mr. Bad"][0]])
var_y = np.var([centered_data["Mr. Good"][1], centered_data["Mr. Bad"][1]])

print(f"\nVariance of age: {var_x}")
print(f"Variance of income: {var_y}")

# Calculate standard deviations
std_x = np.sqrt(var_x)
std_y = np.sqrt(var_y)

print(f"\nStandard deviation of age: {std_x}")
print(f"Standard deviation of income: {std_y}\n")

# Normalize the data (variance one)
normalized_data = {key: [value[0]/std_x, value[1]/std_y] for key, value in centered_data.items()}

for key, value in normalized_data.items():
    print(f"Normalized coordinates for {key}: {value}")

# Plotting
fig, ax = plt.subplots(1, 2, figsize=(6, 3))

# Original data plot
for key, value in data.items():
    ax[0].scatter(value[0], value[1], label=key, alpha=0.6)
ax[0].set_title("Original data")
ax[0].set_xlabel("Age (years)")
ax[0].set_ylabel("Income (thousands of $)")
ax[0].grid(True)
ax[0].set_xlim(0, 50)
ax[0].set_ylim(0, 50)

# Normalized data plot
for key, value in normalized_data.items():
    ax[1].scatter(value[0], value[1], label=key, alpha=0.6)
ax[1].set_title("Normalized data")
ax[1].set_xlabel("Age (normalized)")
ax[1].set_ylabel("Income (normalized)")
ax[1].grid(True)

# Add a single legend for the entire figure
handles, labels = ax[0].get_legend_handles_labels()
fig.legend(handles, labels, loc='upper center', bbox_to_anchor=(0.5, 0.04), ncol=3)

plt.title("Example of Invariance and Normalization")
plt.tight_layout()
plt.savefig('output/invariance_and_normalization.png')
plt.show()

# 3. Differentiable programming (35 points)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch

# Parameter affecting the shape of the quadratic function
p_f = 0.5  

# Quadratic function definition
def f(x, y):
    """Compute the value of the function for given x and y."""
    return (p_f*x)**2 + y**2 + p_f*x*y

def gradient_descent_plot(optimizer_type, p_x, p_y, r, n_iter):
    """
    Plot the contour of the function and the gradient descent path.
    
    Parameters:
    - p_x, p_y: Lists containing x and y coordinates of the gradient descent path.
    - r: Range for plotting.
    - n_iter: Number of iterations.
    """

    plt.figure(figsize=(4, 4))

    # Generate meshgrid for contour plot
    x = np.linspace(-r, r, 50)
    y = np.linspace(-r, r, 50)
    X, Y = np.meshgrid(x, y)
    Z = f(X, Y)
    
    # Draw contour lines and fill
    contours = plt.contour(X, Y, Z, [0.01, 0.05, 0.1, 0.5, 1.], colors='grey')
    plt.clabel(contours, inline=True, fontsize=6)
    plt.imshow(Z, extent=[-r, r, -r, r], origin='lower', cmap='RdGy', alpha=0.5)

    # Mark the optimum point
    plt.plot(0, 0, 'x', c='k')

    # Plot arrows showing the gradient descent path
    for i in range(n_iter):
        plt.arrow(p_x[i], p_y[i], p_x[i+1]-p_x[i], p_y[i+1]-p_y[i], width=.005, head_width=.045, head_length=.025, length_includes_head=True, fc='b', ec='b', zorder=10)
    
    # Save plot
    plt.title(f'Gradient Descent with {optimizer_type}')
    plt.tight_layout()
    plt.savefig(f'output/gradient_descent_{optimizer_type}.png')
    plt.show()

def perform_gradient_descent(optimizer_type, eta, n_iter, r):
    """
    Perform gradient descent optimization and store the path.
    
    Parameters:
    - optimizer_type: Type of optimizer to use ("SGD" or "Adam").
    - eta: Learning rate.
    - n_iter: Number of iterations.
    - r: Range for initial point.
    """
    # Initialize starting point
    x = torch.ones(1, requires_grad=True)
    y = torch.ones(1, requires_grad=True)
    with torch.no_grad():
        x *= 0.9*r
        y *= 0.8*r

    # Lists to store path of gradient descent
    p_x = [x.item()]
    p_y = [y.item()]

    # Choose optimizer based on input
    if optimizer_type == "SGD":
        optimizer = torch.optim.SGD([x, y], lr=eta)
    elif optimizer_type == "Adam":
        optimizer = torch.optim.Adam([x, y], lr=eta)
    else:
        raise ValueError("Unsupported optimizer type")

    # Gradient descent loop
    for i in range(n_iter):
        optimizer.zero_grad()
        f(x, y).backward()
        optimizer.step()
        p_x.append(x.item())
        p_y.append(y.item())

    # Plot the results of gradient descent
    gradient_descent_plot(optimizer_type, p_x, p_y, r, n_iter)

# Range for plotting the function
r = 1.

# Perform and plot gradient descent using SGD optimizer
perform_gradient_descent("SGD", eta = 0.5, n_iter = 15, r = r)

# Perform and plot gradient descent using Adam optimizer
perform_gradient_descent("Adam", eta = 0.5, n_iter = 15, r = r)